In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from statistics import mean
from datetime import datetime, timedelta

In [2]:
# Загружаем данные датафрейма в переменную data
data = pd.read_csv('movie_bd_v5.xls')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1330,tt0371606,150000000,314432665,Chicken Little,Zach Braff|Garry Marshall|Don Knotts|Patrick S...,Mark Dindal,"When it comes to saving the world, it helps to...",When the sky really is falling and sanity has ...,81,Animation|Family|Comedy,Walt Disney Pictures|Walt Disney Feature Anima...,11/4/2005,5.6,2005
625,tt0942385,92000000,188072649,Tropic Thunder,Ben Stiller|Robert Downey Jr.|Jack Black|Jay B...,Ben Stiller,The movie they think they're making... isn't a...,"Vietnam veteran 'Four Leaf' Tayback's memoir, ...",107,Action|Comedy,DreamWorks SKG|Goldcrest Pictures|Red Hour Fil...,8/9/2008,6.4,2008
1041,tt1142977,39000000,81491068,Frankenweenie,Charlie Tahan|Winona Ryder|Catherine O'Hara|Ma...,Tim Burton,The electrifying dog is back from beyond the g...,Young Victor conducts a science experiment to ...,87,Animation|Comedy|Family,Walt Disney Pictures|Tim Burton Animation Comp...,10/4/2012,6.5,2012
1754,tt0457433,60795000,73090611,Perfect Stranger,Halle Berry|Bruce Willis|Giovanni Ribisi|Richa...,James Foley,How Far Would You Go To Keep A Secret?,A journalist goes undercover to ferret out bus...,109,Crime|Drama|Mystery|Thriller,Columbia Pictures|Revolution Studios,4/12/2007,5.3,2007
491,tt1320239,10000000,4349187,Burke & Hare,Simon Pegg|Andy Serkis|Isla Fisher|Tom Wilkins...,John Landis,No Job Too Small. No Body Too Big. No Question...,"Two 19th-century opportunists (Simon Pegg, And...",91,Comedy|Thriller,Ealing Studios|Fragile Films|Quickfire Films|P...,10/29/2010,6.0,2010


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

# Рассчитываем прибыль фильма: (profit = revenue - budget)
data['profit'] = data['revenue'] - data['budget']


def string_conversion(row, dict_name):
    """Функция получает строку разбивает их по символу '|' 
    и добавляет в словарь dict_name при этом считая
    количество повторений"""
    
    items = row.split('|')   # Разбиваем строку по символу
    
    for item in items:                # Заполняем словарь
        if item in dict_name:
            dict_name[item] += 1
        else:
            dict_name.setdefault(item, 1)
            
            
def revenue_actors_count(row):
    """Функция получает в качестве значений строку.
    Строка с актёрами разбивается по символу '|' и добавляется
    в словарь в виде ключа, где значением является 
    сумма сборов"""
    
    revenue = row[0]
    actors = row[1].split('|')
    
    for actor in actors:
        if actor in revenue_actors:
            revenue_actors[actor] += revenue
        else:
            revenue_actors.setdefault(actor, revenue)
            
            
def original_title_letter(row):
    """Функция original_title_letter получает в качестве
    значения название фильма и производителя, после чего
    считает количество символов в названии и заполняет
    словарь letter_count где в качестве ключа выступает
    название компании, а в качестве значения количество
    символов в названии фильма."""
    
    original_title = len(row[0])
    production_companies_list = row[1].split('|')
    
    for production_companie in production_companies_list:
        if production_companie in letter_count:
            letter_count[production_companie].append(original_title)
        else:
            letter_count.setdefault(production_companie, list([original_title]))
            

def dict_mean_count(dict_mean):
    """Функция dict_mean_count получает в качестве аргумента
    словарь и для каждого ключа высчитывает среднее
    его значения, после чего возвращает обновлённый словарь"""
    
    for key, value in dict_mean.items():
        dict_mean[key] = mean(value)
    return dict_mean


def number_of_words(row):
    """Функция number_of_words получает в качестве
    значения описание фильма и производителя, после чего
    считает количество слов в описании и заполняет
    словарь word_count где в качестве ключа выступает
    название компании, а в качестве значения количество
    слов в описании фильма."""
    
    overview = len(row[0].split(' '))
    production_companies_list = row[1].split('|')
    
    for production_companie in production_companies_list:
        if production_companie in word_count:
            word_count[production_companie].append(overview)
        else:
            word_count.setdefault(production_companie, list([overview]))
            
            
def actor_duets_count(names):
    """Функция actor_duets_count получает в качестве
    значения имена актёров и формирует из них пары для
    добавления в лист actor_duets_list"""
    
    name_number = 0
    next_name_number = 1
    names = names.split('|')
    names.sort()
    
    while name_number < len(names)-1:
        actor_duets_list.append(names[name_number] + ' & ' + names[next_name_number])
        next_name_number += 1
        
        if next_name_number == len(names):
            name_number += 1
            next_name_number = name_number + 1

# 1. У какого фильма из списка самый большой бюджет?

In [5]:
# выводим фильм с самым большим бюджетом
data[data['budget'] == data['budget'].max()]

# Записываем результат в словарь с ответами
answers['1'] = '723, Pirates of the Caribbean: On Stranger Tides (tt1298650)'
answers['1'] # +

'723, Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [6]:
# выводим самый длиный фильм в датафрейме
data[data['runtime'] == data['runtime'].max()]

# Записываем результат в словарь с ответами
answers['2'] = '1157, Gods and Generals (tt0279111)'
answers['2'] # +

'1157, Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [7]:
# выводим самый короткий фильм в датафрейме
data[data['runtime'] == data['runtime'].min()]

# Записываем результат в словарь с ответами
answers['3'] = '768, Winnie the Pooh (tt1449283)'
answers['3'] # +

'768, Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [8]:
# Расчитываем среднюю длительность фильмов
mean_time_films = round(data['runtime'].mean())

# Записываем результат в словарь с ответами
answers['4'] = mean_time_films
answers['4'] # +

110

# 5. Каково медианное значение длительности фильмов? 

In [9]:
# Расчитываем медиану фильмов
median_films = data['runtime'].median()

# Записываем результат в словарь с ответами
answers['5'] = median_films
answers['5'] # +

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [10]:
# Отфильтровываем самый прибыльный фильм
data[data['profit'] == data['profit'].max()]

# Записываем результат в словарь с ответами
answers['6'] = '239, Avatar (tt0499549)'
answers['6'] # +

'239, Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [11]:
# Отфильтровываем самый убыточный фильм
data[data['profit'] == data['profit'].min()]

# Записываем результат в словарь с ответами
answers['7'] = '1245, The Lone Range (tt1210819)'
answers['7'] # +

'1245, The Lone Range (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [12]:
# Отфильтровываем фильмы где объём сборов оказался выше бюджета
count_films = len(data.query('revenue > budget'))

# Записываем результат в словарь с ответами
answers['8'] = count_films
answers['8'] # +

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [13]:
# Отфильтровываем самые кассовые фильмы 2008 года
all_films_2008 = data.query('release_year == 2008')
all_films_2008[all_films_2008['revenue'] == all_films_2008['revenue'].max()]

# Записываем результат в словарь с ответами
answers['9'] = '599, The Dark Knight (tt0468569)'
answers['9'] # +

'599, The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [14]:
# Отфильтровываем самые убыточные фильмы 2012 - 2014 года. 
all_filams_12_14 = data.query('2012 <= release_year <= 2014')
all_filams_12_14[all_filams_12_14['profit'] == all_filams_12_14['profit'].min()]

# Записываем результат в словарь с ответами
answers['10'] = '1245, The Lone Ranger (tt1210819)'
answers['10'] # +

'1245, The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

ВАРИАНТ 1

In [15]:
# Создаём пустой словарь для хранения жанрав фильмов
genres_films = dict()
           
# Передаём данные и словарь в функцию string_conversion для заполения 
# словаря genres_films и подсчёта вхождений жанров фильмов в датасет
data['genres'].apply(lambda row: string_conversion(row, genres_films))

# Преобразуем словарь в финальный датасет
final_series = pd.Series(genres_films)
final_series.sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['11'] = final_series.sort_values(ascending=False).index[0]
answers['11'] # +

'Drama'

ВАРИАНТ 2

In [16]:
# Создаём пустой лист для хранения жанрав фильмов
genres_films = list()

# Передаём данные в lambda функцию для преобразования строки
data['genres'].apply(lambda row: [genres_films.append(genre) for genre in row.split('|')])

# Считаем повторяющиеся значения в листе и 
# преобразуем лист в словарь
# На основе словаря создаём датасет
genres_films = pd.Series(Counter(genres_films))

# Фильтуем по убыванию
genres_films.sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['11'] = final_series.sort_values(ascending=False).index[0]
answers['11'] # +

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

ВАРИАНТ 1

In [17]:
# Создаём пустой словарь для хранения жанрав фильмов
genres_films = dict()

# Отфильтровываем фильмы прибыль которых больше 0
profit_films = data.query('profit > 0')

# Передаём данные и словарь в функцию string_conversion для заполения 
# словаря genres_films и подсчёта вхождений жанров фильмов в датасет
profit_films['genres'].apply(lambda row: string_conversion(row, genres_films))

# Преобразуем словарь в датасет
genres_films = pd.Series(genres_films)

# Фильтуем по убыванию
genres_films.sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['12'] = final_series.sort_values(ascending=False).index[0]
answers['12'] # +

'Drama'

ВАРИАНТ 2

In [18]:
# Создаём пустой лист для хранения жанрав фильмов
genres_films = list()

# Отфильтровываем фильмы прибыль которых больше 0
profit_films = data.query('profit > 0')

# Передаём данные в lambda функцию для преобразования строки
profit_films['genres'].apply(lambda row: [genres_films.append(genre) for genre in row.split('|')])

# Считаем повторяющиеся значения в листе и 
# преобразуем лист в словарь
genres_films = pd.Series(Counter(genres_films))

# Фильтуем по убыванию
genres_films.sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['12'] = final_series.sort_values(ascending=False).index[0]
answers['12'] # +

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [19]:
# Группируем датасет по режиссерам и суммируем кассовые сборы
data.groupby('director')['revenue'].sum().sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['13'] = data.groupby('director')['revenue'].sum().sort_values(ascending=False).index[0]
answers['13'] # +

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

ВАРИАНТ 1

In [20]:
# Создаём пустой ckjdfhm для хранения имён режисёров
action_director = dict()

# Фильтруем фильмы в жанрах которых присутствует Action
action_films = data[data['genres'].str.contains('Action')]

# Передаём данные и словарь в функцию string_conversion для заполения 
# словаря action_director и подсчёта вхождений жанров фильмов в датасет
action_films['director'].apply(lambda row: string_conversion(row, action_director))

# Преобразуем словарь в датасет
director_films = pd.Series(action_director)

# Фильтуем по убыванию
director_films.sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['14'] = director_films.sort_values(ascending=False).index[0]
answers['14'] # +

'Robert Rodriguez'

ВАРИАНТ 2

In [21]:
# Создаём пустой лист для хранения имён режисёров
action_director = list()

# Фильтруем фильмы в жанрах которых присутствует Action
action_films = data[data['genres'].str.contains('Action')]

# Передаём данные в lambda функцию для преобразования строки
action_films['director'].apply(lambda directors: [action_director.append(director) for director in directors.split('|')])

# Считаем повторяющиеся значения в листе и 
# преобразуем лист в датасет
director_films = pd.Series(Counter(action_director))

# Фильтуем по убыванию
director_films.sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['14'] = director_films.sort_values(ascending=False).index[0]
answers['14'] # +

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [22]:
# Создаём пустой cловарь для хранения имён режисёров
revenue_actors = dict()

# Отфильтровываем фильмы выпущенные в 2012 году
films_2012 = data.query('release_year == 2012')

# Передаём данные и словарь в функцию revenue_actors_count для заполения 
# словаря revenue_actors и подсчитывае самые высокие коссовые сборы
films_2012[['revenue', 'cast']].apply(lambda row: revenue_actors_count(row), axis=1)

# Преобразуем словарь в датасет
final_series = pd.Series(revenue_actors)

# Фильтуем по убыванию
final_series.sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['15'] = final_series.sort_values(ascending=False).index[0]
answers['15'] # +

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [23]:
# Создаём пустой лист для хранения имён актёров
actors_list = list()

# Отфильтровываем высокобюджетные фильмы
top_budget_films = data[data['budget'] > data['budget'].mean()]

# Передаём данные в lambda функцию для преобразования строки
top_budget_films['cast'].apply(lambda row: [actors_list.append(actor) for actor in row.split('|')])

# Считаем повторяющиеся значения в листе и 
# преобразуем лист в датасет
final_series = pd.Series(Counter(actors_list))

# Фильтуем по убыванию
final_series.sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['16'] = final_series.sort_values(ascending=False).index[0]
answers['16'] # +

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [24]:
# Создаём пустой лист для хранения жанров
genres_list = list()

# Отфильтровываем фильмы где снимался Nicolas Cage
nc_films = data[data['cast'].str.contains('Nicolas Cage')]

# Передаём данные в lambda функцию для преобразования строки
nc_films['genres'].apply(lambda row: [genres_list.append(genres) for genres in row.split('|')])

# Считаем повторяющиеся значения в листе и 
# преобразуем лист в датасет
final_series = pd.Series(Counter(genres_list))

# Фильтуем по убыванию
final_series.sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['17'] = final_series.sort_values(ascending=False).index[0]
answers['17'] # +

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [25]:
# Отфильтровываем фильмы Paramount Pictures
pp_films = data[data['production_companies'].str.contains('Paramount Pictures')]

# Ищем самый убыточный фильм
pp_films[pp_films['profit'] == pp_films['profit'].min()]

# Записываем результат в словарь с ответами
answers['18'] = '925, K-19: The Widowmaker (tt0267626)'
answers['18'] # +

'925, K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [26]:
# Группируем датасет по годам и суммируем по кассовым сборам
data.groupby('release_year')['revenue'].sum().sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['19'] = 2015
answers['19'] # +

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [27]:
# Отфильтровываем фильмы Warner Bros
wb_films = data[data['production_companies'].str.contains('Warner Bros')]

# Группируем датасет по годам и суммируем по прибыли
wb_films.groupby('release_year')['profit'].sum().sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['20'] = 2014
answers['20'] # +

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [28]:
# Создаём столбец month состоящий из месяцев
data['month'] = data['release_date'].apply(lambda row: int(row.split('/')[0]))

# групперуем по месяцам и подсчитываем значения
data.groupby('month')['imdb_id'].count().sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['21'] = 'Сентябрь'
answers['21'] # +

'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [29]:
# Фильтруем столбец month летними мясецами
summer_films = data.query('month in [6, 7, 8]')

# Подсчитываем количество фильмов
answers['22'] = len(summer_films)
answers['22'] # +

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [30]:
# Создаём пустой лист для хранения имён режиссёров
director_list = list()

# Фильтруем столбец month Зимним мясецами
winter_films = data.query('month in [12, 1, 2]')

# Передаём данные в lambda функцию для преобразования строки
winter_films['director'].apply(lambda row: [director_list.append(director) for director in row.split('|')])

# Считаем повторяющиеся значения в листе и 
# преобразуем лист в датасет
final_series = pd.Series(Counter(director_list))

# Фильтуем по убыванию
final_series.sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['23'] = final_series.sort_values(ascending=False).index[0]
answers['23'] # +

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [31]:
# Создаём пустой словарь для хранения имён режиссёров
letter_count = dict()

# Передаём данные и словарь в функцию original_title_letter для заполения 
# словаря letter_count и подсчитываем количество символов с строке
data[['original_title', 'production_companies']].apply(lambda row: original_title_letter(row), axis=1)

# Передаём словарь в функцию dict_mean_count и преобразуем полученный
# словарь в датасет
final_series = pd.Series(dict_mean_count(letter_count))

# Фильтуем по убыванию
final_series.sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['24'] = final_series.sort_values(ascending=False).index[0]
answers['24'] # +

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [32]:
# Создаём пустой словарь для хранения имён режиссёров
word_count = dict()


# Передаём данные и словарь в функцию number_of_words для заполения 
# словаря word_count и подсчитываем количество слов с строке
data[['overview', 'production_companies']].apply(lambda row: number_of_words(row), axis=1)

# Передаём словарь в функцию dict_mean_count и преобразуем полученный
# словарь в датасет
final_series = pd.Series(dict_mean_count(word_count))

# Фильтуем по убыванию
final_series.sort_values(ascending=False).head(1)

# Записываем результат в словарь с ответами
answers['25'] = final_series.sort_values(ascending=False).index[0]
answers['25'] # +

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [33]:
# Фильтруе фатасет по столбцу vote_average и получаем фильмы которые
# вошли в 1% лучших по рейтингу
data[data['vote_average'] > data.quantile(0.99, numeric_only=True)['vote_average']]

# Записываем результат в словарь с ответами
answers['26'] = '9, Inside Out (tt2096673), 599, The Dark Knight (tt0468569), 1191, 12 Years a Slave (tt2024544)'
answers['26'] # +

'9, Inside Out (tt2096673), 599, The Dark Knight (tt0468569), 1191, 12 Years a Slave (tt2024544)'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [34]:
# Создаём пустой лист для хранения пар актёров
actor_duets_list = list()

# Передаём строку с миенами актёров функции actor_duets_count
data['cast'].apply(actor_duets_count)

# Считаем повторяющиеся значения в листе и 
# преобразуем лист в датасет
final_series = pd.Series(Counter(actor_duets_list))

# Фильтуем по убыванию
final_series.sort_values(ascending=False).head(10)

# Записываем результат в словарь с ответами
answers['27'] = 'Daniel Radcliffe & Rupert Grint'
answers['27'] # +

'Daniel Radcliffe & Rupert Grint'

# Submission

In [35]:
# Ответы на вопросы
answers

{'1': '723, Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '1157, Gods and Generals (tt0279111)',
 '3': '768, Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107.0,
 '6': '239, Avatar (tt0499549)',
 '7': '1245, The Lone Range (tt1210819)',
 '8': 1478,
 '9': '599, The Dark Knight (tt0468569)',
 '10': '1245, The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': '925, K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': '9, Inside Out (tt2096673), 599, The Dark Knight (tt0468569), 1191, 12 Years a Slave (tt2024544)',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [36]:
# количество ответов совподает с количеством вопросов
27 == len(answers)

True